# Interpolação Polinomial em Casos Diarios da Covid 19 nos Estados Do Brasil e Apresentação de Dados em diferentes regiões do país


Logo abaixo você vai ver a apresentação de dados e a interpolação polinomial nos dados das médias de casos diários da covid 19 em todos os estados do Brasil, como a apresentação das taxas de mortalidade número de mortos e de casos acumulados nas diferentes regiões do país.  

# Casos Diários de Covid nos Estados do Brasil e previsão através da interpolação polinomial. 

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from datetime import datetime

plt.style.use('fivethirtyeight')
def ploting(y,z,media_de_semana,media_tres_dias,k,ind,casos):

    
    plt.figure(casos + ind,figsize=(12,6))
    plt.title(casos + ind)
    plt.plot_date(y,casos_dia,'-',label=casos + "dia")
    plt.plot_date(y,media_de_semana,'-',label="média móvel de 7 dias")
    plt.plot_date(y,media_tres_dias,'-', label="média movel de 3 dias")
    plt.plot_date(z,k,'-',label='previsão')
    plt.legend()
    plt.show()

prec=15
db = pd.read_csv('../input/corona-virus-brazil/brazil_covid19.csv')
regioes=[]
datas=[]
casos_brasil = np.zeros(prec)
casos_dia=[]
cases_sul=[0]
casos_geral_brasil= []
casos=db['cases']
mortes=db['deaths']
for i in range(len(casos)):
    if not(db['state'][i] in regioes):
        regioes.append(db['state'][i])

for ind in regioes:

    model=LinearRegression()
    casos_dia=[]
    cases_sul=[0]
    for r in range(len(db['date'])):
        if not(db['date'][r] in datas):
            datas.append(db['date'][r]) 
    for r in range(len(casos)):
        if db['state'][r]== ind:
            cases_sul.append(casos[r])
    for r in range(1,len(cases_sul)):
        casos_dia.append(cases_sul[r]-cases_sul[r-1])

    media_de_semana=[0,0,0,0,0,0]
    media_tres_dias=[0,0]

    for r in range(6,len(casos_dia)):
        media_de_semana.append((casos_dia[r-6]+casos_dia[r-5]+casos_dia[r-4]+casos_dia[r-3]+casos_dia[r-2]+casos_dia[r-1]+casos_dia[r])/7)
    for r in range(2,len(casos_dia)):
        media_tres_dias.append((casos_dia[r-2]+casos_dia[r-1]+casos_dia[r])//3)

    day_average_int,covariance=np.polyfit(np.linspace(0,len(media_de_semana),len(media_de_semana)),media_de_semana,deg=4,cov=True)
    day_average_int=np.poly1d(day_average_int)

    x=np.linspace(0,len(media_de_semana)+prec,len(media_de_semana)+prec)
    y=[mdates.datestr2num(u) for u in datas ]
    z=np.linspace(y[0],y[-1]+prec,len(y)+prec)
    z2=np.linspace(0,len(media_de_semana),len(media_de_semana)).reshape(-1,1)
    x_ = PolynomialFeatures(degree=4, include_bias=False).fit_transform(z2)
    model.fit(x_ , (media_de_semana))
    r_sq = model.score(x_, np.array(media_de_semana))
    print(str(r_sq)+' Score do Estado '+ ind )
    k=np.array(day_average_int(x))
    print('Estado '+ ind)
    print('Media de 7 dias esperada para os dias no Estado: '+ind)
    for r in range(-prec,0):
        casos_brasil[r]+=k[r]
        print(mdates.num2date(z[r]).strftime('%d-%m-%Y')+'     '+ str((k[r])))
    ploting(y,z,media_de_semana,media_tres_dias,k,ind,"Casos ")
print('Geral casos Brasil')
for r in range(-prec,0):
    print(mdates.num2date(z[r]).strftime('%d-%m-%Y')+'     '+ str((casos_brasil[r])))

# Mortes Diárias de Covid nos Estados do Brasil e previsão através da intepolação polinomial.

In [ ]:
mortes_brasil = np.zeros(prec)
for ind in regioes:

    model=LinearRegression()
    casos_dia=[]
    cases_sul=[0]
    for r in range(len(db['date'])):
        if not(db['date'][r] in datas):
            datas.append(db['date'][r]) 
    for r in range(len(casos)):
        if db['state'][r]== ind:
            cases_sul.append(mortes[r])
    for r in range(1,len(cases_sul)):
        casos_dia.append(cases_sul[r]-cases_sul[r-1])

    media_de_semana=[0,0,0,0,0,0]
    media_tres_dias=[0,0]

    for r in range(6,len(casos_dia)):
        media_de_semana.append((casos_dia[r-6]+casos_dia[r-5]+casos_dia[r-4]+casos_dia[r-3]+casos_dia[r-2]+casos_dia[r-1]+casos_dia[r])/7)
    for r in range(2,len(casos_dia)):
        media_tres_dias.append((casos_dia[r-2]+casos_dia[r-1]+casos_dia[r])//3)

    day_average_int,covariance=np.polyfit(np.linspace(0,len(media_de_semana),len(media_de_semana)),media_de_semana,deg=4,cov=True)
    day_average_int=np.poly1d(day_average_int)

    x=np.linspace(0,len(media_de_semana)+prec,len(media_de_semana)+prec)
    y=[mdates.datestr2num(u) for u in datas ]
    z=np.linspace(y[0],y[-1]+prec,len(y)+prec)
    z2=np.linspace(0,len(media_de_semana),len(media_de_semana)).reshape(-1,1)

    x_ = PolynomialFeatures(degree=4, include_bias=False).fit_transform(z2)
    model.fit(x_ , (media_de_semana))
    r_sq = model.score(x_, np.array(media_de_semana))
    print(str(r_sq)+' Score Mortes do Estado '+ ind )
    k=day_average_int(x)
    print('Estado '+ ind)
    print('Media de mortes de 7 dias esperada para os dias no Estado: '+ind)
    for r in range(-prec,0):
        mortes_brasil[r]+=k[r]
        print(mdates.num2date(z[r]).strftime('%d-%m-%Y')+'      '+ str((k[r])))
    ploting(y,z,media_de_semana,media_tres_dias,k,ind,"Mortes ")
print('Mortes em geral esperadas no Brasil')
for r in range(-prec,0):
    print(mdates.num2date(z[r]).strftime('%d-%m-%Y')+'      '+ str((mortes_brasil[r])))

Acima é possível visualizar os dados da Covid nos estados do Brasil e uma interpolação polinomial de grau 5, foi postado anteriormente usando grau 4, pórem para alguns estados a previsão foi muito contraria aos números reais da pandemia. A intepolação foi feita na média móvel de 7 dias em cada um deles, com o objetivo de diminuir outliers. A variavel preci determina a quantidade de dias da previsão do modelo de interpolação. Sei que este modelo é fraco pois não considera questões como movimentação de pessoas e a taxa de transmisibilidade da doença. Existem alguns estados com números esquisitos que eu tenha percebido são SC,CE e RN que podem ter sido causados por correções dos dados das secretarías ou de atrasos na divulgação, espero que estes dados não tenham grandes mudanças na média móvel.Os dados com os casos por dia foram obtidos a partir da diferença da quantidade de casos registrados de um dia para o outro, por isso podem ocorrer oscilações inesperadas dos dados. Lembrando que quanto mais longe da data do valor obtido maior a possibilidade de variações. Ainda assim, em alguns estados algumas previsões não fazem sentido e uma previsão de menor tempo pode ser mais precisa.Tambem foram feitas interpolações para o numero de mortes diárias em cada Estado do Brasil.
Outro problema do modelo é que ele admite uma função continua e no periodo do estudo com derivada positiva e não toma em conta a sazonalidade da aparição dos dados que é clara, nos finais de semana o número de casos e mortes é menor que em dias da semana, então estes valores podem ser admitidos como as possíveis médias moveis para os proximos dias em relação a casos e mortes. 


# Dados Por regiões.
Agora podemos analisar os dados por região e os dados acumulados por região para isso é construida uma função para preparar os dados

In [ ]:
def append_region(regiao, db, ind):
    lista = []
    val = 0
    teste = False
    for r in range(len(db[ind])):
        if db['region'][r] == regiao:
            val += db[ind][r]
            teste = True
        elif teste:
            lista.append(val)
            val = 0
            teste = False
    return lista

Agora construimos as listas com cada um dos valores por data. 

In [ ]:
casos_sul = append_region('Sul', db, 'cases')
casos_sudeste = np.array(append_region('Sudeste', db, 'cases'))
casos_norte = np.array(append_region("Norte", db, 'cases'))
casos_nordeste = np.array(append_region("Nordeste", db, 'cases'))
casos_centro = np.array(append_region("Centro-Oeste", db, 'cases'))
casos_sul.insert(0, 0)
casos_sul = np.array(casos_sul)

Plotamos agora os graficos com o número de casos acumulados por região.

In [ ]:
x=[mdates.datestr2num(u) for u in datas ]
plt.figure("Casos",figsize=(12,6))

plt.title("Casos Acumulados por regiões")
plt.plot_date(x, casos_sul, '-', label="Sul")
plt.plot_date(x, casos_sudeste, '-', label="Sudeste")
plt.plot_date(x, casos_centro, '-', label="Centro")
plt.plot_date(x, casos_nordeste, '-', label="Nordeste")
plt.plot_date(x, casos_norte, '-', label="Norte")

plt.xlabel("Dias")
plt.ylabel("Casos Acumulados")

plt.legend()
plt.show()

Podemos tambem analisar o número de casos em geral no brasil, que sera igual a soma do número de casos acumulados no brasil

In [ ]:
casos_geral = casos_sul+casos_sudeste+casos_centro+casos_nordeste+casos_norte

plt.figure(figsize = (12,6))
plt.title('Casos Acumulados Brasil')
plt.plot_date(x, casos_geral, '-', label="Brasil")

plt.xlabel("Dias")
plt.ylabel("Casos Acumulados")

plt.legend()
plt.show()

Podemos agora analisar o número de óbitos acumulados causados pela Covid-19 por região para isso organizamos os dados. 

In [ ]:
mortes_sul = append_region('Sul', db, 'deaths')
mortes_sudeste = np.array(append_region('Sudeste', db, 'deaths'))
mortes_norte = np.array(append_region('Norte', db, 'deaths'))
mortes_nordeste = np.array(append_region('Nordeste', db, 'deaths'))
mortes_centro = np.array(append_region('Centro-Oeste', db, 'deaths'))
mortes_sul.insert(0, 0)
mortes_sul = np.array(mortes_sul)

Em seguida plotamos os dados de mortes

In [ ]:
plt.figure('Mortes',figsize=(12,6))

plt.title("Mortes Acumuladas por Região")
plt.plot_date(x, mortes_sul, '-', label="Sul")
plt.plot_date(x, mortes_sudeste, '-', label="Sudeste")
plt.plot_date(x, mortes_centro, '-', label="Centro")
plt.plot_date(x, mortes_nordeste, '-', label="Nordeste")
plt.plot_date(x, mortes_norte, '-', label="Norte")

plt.xlabel("Dias")
plt.ylabel("Mortes Acumuladas")

plt.legend()
plt.show()

Podemos agora analisar o número de mortes acumuladas no Brasil 

In [ ]:
mortes_geral = mortes_sul+mortes_sudeste+mortes_centro+mortes_nordeste+mortes_norte
plt.figure(figsize=(12,6))
plt.title("Mortes acumuladas Brasil")
plt.plot_date(x, mortes_geral, '-', label="Brasil")

plt.xlabel("Dias")
plt.ylabel("Mortes Acumuladas")

plt.legend()
plt.show()


Agora com os dados de mortes e casos podemos obter a taxa de mortalidade em relação ao tempo para isso contruimos uma função para obter cada taxa por região.

In [ ]:
def taxa_mortalidade(casos_acumulados, mortes_acumuladas):
    lista = []
    for r in range(len(casos_acumulados)):
        try:
            lista.append(mortes_acumuladas[r]/casos_acumulados[r])
        except:
            lista.append(0)
    return lista

Note que algns valores de casos podem ser zero por isso substituímos pela taxa como sendo zero neste caso específico. 
Em seguida usaremos a função para obter os dados da taxa de mortalidade. Lembrando que determinamos a taxa de mortalidade como:
*Número de mortos acumulado / Número de casos acumulados*

In [ ]:
taxa_de_mortalidade_sul = taxa_mortalidade(casos_sul, mortes_sul)
taxa_de_mortalidade_norte = taxa_mortalidade(casos_norte, mortes_norte)
taxa_de_mortalidade_sudeste = taxa_mortalidade(casos_sudeste, mortes_sudeste)
taxa_de_mortalidade_Nordeste = taxa_mortalidade(casos_nordeste, mortes_nordeste)
taxa_de_mortalidade_Centro = taxa_mortalidade(casos_centro, mortes_centro)
taxa_de_mortalidade_Brasil = taxa_mortalidade(casos_geral,mortes_geral)

Agora plotamos os valores da taxa de mortalidade em relação ao tempo

In [ ]:
plt.figure('Taxa',figsize=(12,6))

plt.title('Taxa de mortalidade por região')
plt.plot_date(x, taxa_de_mortalidade_sul, '-', label="Sul")
plt.plot_date(x, taxa_de_mortalidade_sudeste, '-', label="Sudeste")
plt.plot_date(x, taxa_de_mortalidade_Centro, '-', label="Centro")
plt.plot_date(x, taxa_de_mortalidade_Nordeste, '-', label="Nordeste")
plt.plot_date(x, taxa_de_mortalidade_norte, '-', label="Norte")
plt.plot_date(x, taxa_de_mortalidade_Brasil,'-',label = "Brasil")
plt.xlabel("Dias")
plt.ylabel("taxa de mortalidade")

plt.legend()
plt.show()

Indo ao site do IBGE  em https://ftp.ibge.gov.br/Estimativas_de_Populacao/Estimativas_2020/POP2020_20201030.pdf podemos retirar de informação a estimativa de pessoas por região e adicionar isto ao grafico e obter o número de mortos por 100 mil habitantes, como o número de casos acumulados por 100 mil habitantes. 

In [ ]:
# dados IBGE 2020 retirados de https://ftp.ibge.gov.br/Estimativas_de_Populacao/Estimativas_2020/POP2020_20201030.pdf
pop_sul=30192315
pop_sudeste=89012240
pop_norte=18672591
pop_centro=16504303
pop_nordeste=57374243
pop_brasil = pop_sul+pop_sudeste+pop_centro+pop_norte+pop_nordeste

mortes_por_100_NE=[100000*u/pop_nordeste for u in mortes_nordeste]
mortes_por_100_SD=[100000*u/pop_sudeste for u in mortes_sudeste]
mortes_por_100_SL=[100000*u/pop_sul for u in mortes_sul]
mortes_por_100_NO=[100000*u/pop_norte for u in mortes_norte]
mortes_por_100_CO=[100000*u/pop_centro for u in mortes_centro]
mortes_por_100 =  [100000*u/pop_brasil for u in mortes_geral]

casos_por_100_NE=[100000*u/pop_nordeste for u in casos_nordeste]
casos_por_100_SD=[100000*u/pop_sudeste for u in casos_sudeste]
casos_por_100_SL=[100000*u/pop_sul for u in casos_sul]
casos_por_100_NO=[100000*u/pop_norte for u in casos_norte]
casos_por_100_CO=[100000*u/pop_centro for u in casos_centro]
casos_por_100 =  [100000*u/pop_brasil for u in casos_geral]

Agora possuimos a variação em relação ao tempo do número de casos por 100 mil habitantes, como também o número de óbitos por 100 mil habitantes

In [ ]:
plt.figure("Morte por 100 mil regiões",figsize=(12,6))
plt.title('Mortes por 100 mil habitantes')
plt.plot_date(x, mortes_por_100_SL, '-', label="Sul")
plt.plot_date(x, mortes_por_100_SD, '-', label="Sudeste")
plt.plot_date(x, mortes_por_100_CO, '-', label="Centro")
plt.plot_date(x, mortes_por_100_NE, '-', label="Nordeste")
plt.plot_date(x, mortes_por_100_NO, '-', label="Norte")
plt.plot_date(x, mortes_por_100,'-',label = 'Brasil')

plt.xlabel('Dias')
plt.ylabel("Mortes por 100 mil  habitantes")
plt.legend()

plt.figure("Casos por 100 mil regiões",figsize=(12,6))

plt.title('Casos por 100 mil habitantes')
plt.plot_date(x, casos_por_100_SL, '-', label="Sul")
plt.plot_date(x, casos_por_100_SD, '-', label="Sudeste")
plt.plot_date(x, casos_por_100_CO, '-', label="Centro")
plt.plot_date(x, casos_por_100_NE, '-', label="Nordeste")
plt.plot_date(x, casos_por_100_NO, '-', label="Norte")
plt.plot_date(x, casos_por_100,'-',label = 'Brasil')

plt.xlabel('Dias')
plt.ylabel('Casos por 100 mil habitantes ')
plt.legend()


plt.show()


Lembrando que sou somente um estudante e estou disposto a aprender mais, sugestões são muito bem vindas.